In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
print(train.shape)
train.head()

In [ ]:
train[['C-deck', 'C-num', 'C-side']] = train['Cabin'].str.split('/', expand=True)
train.head()

In [ ]:
train1 = train 
train1[['gggg','pp']] = train['PassengerId'].str.split('_', expand=True).astype(int)
train1['Count'] = 0
train1.head()

In [ ]:
# This code segment adds Count variable, which counts the group size for training set

for i in set(train1['pp']):
    for index, row in train1.iterrows():
        if row['pp'] == i:
            train1.loc[train1['gggg'] == row['gggg'], 'Count'] = i
            #if index > 10:
            #    break
                
train1.head()

In [ ]:
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
print(test.shape)
test[['C-deck', 'C-num', 'C-side']] = test['Cabin'].str.split('/', expand=True)
test.head()

In [ ]:
test1 = test 
test1[['gggg','pp']] = test['PassengerId'].str.split('_', expand=True).astype(int)
test1['Count'] = 0
test1.head()

In [ ]:
# This code segment adds Count variable, which counts the group size for test set

for i in set(test1['pp']):
    for index, row in test1.iterrows():
        if row['pp'] == i:
            test1.loc[test1['gggg'] == row['gggg'], 'Count'] = i
            #if index > 10:
            #   break
                
test1.head(10)

In [ ]:
#Check and print if any group(PassengerId) has been split in train and test sets 
for i in set(test1['gggg']):
    for j in set(train1['gggg']):
        if i == j:
            print(test1['gggg'])

In [ ]:
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")
submission.head(2)

In [ ]:
print(train1.isna().sum(),'\n')
print(test1.isna().sum())

In [ ]:
from sklearn.impute import SimpleImputer

imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]
STRATEGY = 'mean'
imputer = SimpleImputer(strategy=STRATEGY )
imputer.fit(train1[imputer_cols])
train1[imputer_cols] = imputer.transform(train1[imputer_cols])
test1[imputer_cols] = imputer.transform(test1[imputer_cols])
train1["HomePlanet"].fillna('Z', inplace=True)
test1["HomePlanet"].fillna('Z', inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_cols = ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP", "C-deck", "C-num", "C-side"]
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] =  LabelEncoder().fit_transform(test[col])
    return train, test

train2 ,test2 = label_encoder(train1,test1 ,label_cols)

In [ ]:
train2.head()

In [ ]:
train3 = train2.drop(["Name" ,"Cabin", "gggg", "pp"] , axis = 1)
test3 = test2.drop(["Name" ,"Cabin", "gggg", "pp"] , axis = 1)
X = train3.drop('Transported' , axis =1 )
y = train3['Transported']

from sklearn.model_selection import train_test_split
x_train , x_val , y_train , y_val = train_test_split(X, y, random_state = 0, test_size =0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(x_train, y_train)

y_pred_val = model.predict(x_val)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
print("\nAccuracy:", "{0:0.2%}".format(accuracy_score(y_val, y_pred_val)))
cm =  confusion_matrix(y_val, y_pred_val)
print("\nF1 Score:","{0:0.2%}".format(f1_score(y_val, y_pred_val)))

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_,)
disp.plot()
plt.show()

In [ ]:
model1 = RandomForestClassifier()   # Train on full data

model1.fit(X, y)

y_pred_test = model1.predict(test3)

In [ ]:
submission['Transported'] = y_pred_test
submission.to_csv('submission.csv', index=False)
submission